In [1]:
import numpy as np

from GOElement import coor_sys, simple_lens, Aperture_screen, rim, global_coord
from EOElement import frequencyList, GaussBeam, GaussBeam_Near, Elliptical_Beam
from EOElement import lens_PO
from EOElement import Spherical_grid
#from CreateSurf import zemax2RSF
SILICON = 3.36
freq = '90GHz'

def beam_para(T,T_angle,var, Type = 'lambda'):
    c = 299792458000 #mm/s
    if Type == 'lambda':
        Lambda = var
    elif Type == 'freq':
        Lambda = c / (var*10**9)
    #T_A = T_angle/190*np.pi
    # Half angluar size of Gaussian beam
    Theta_c = np.sqrt(20*T_angle**2/np.abs(T)*np.log10(np.exp(1)))
    w_0 = Lambda/np.pi/(Theta_c*np.pi/180)

    return w_0, Theta_c

# define frequency, feedhorn and elliptical feedhorn
# Elliptical Gaussian beam parameters

Feed_list = {'90GHz': {'freq': 90,
                       'Ellip_Taper': -2.1714724,
                       'T_angle_x' : 15.4270683,
                       'T_angle_y' : 20.2798893,
                       'Gauss_Taper':  -20*np.log10(np.exp(1)),
                       'Gauss_Tangle': 30.8,
                       'beam_radius': '1.972 mm',
                       'phase_front_radius': '0 mm'},
            '150GHz': {'freq':150,
                       'Ellip_Taper': -2.1714724,
                       'T_angle_x' : 10.1161095,
                       'T_angle_y' : 11.8095882,
                       'Gauss_Taper': -20*np.log10(np.exp(1)),
                       'Gauss_Tangle':20.232219,
                       'beam_radius': '1.802 mm',
                       'phase_front_radius': '0 mm'},
            '220GHz': {'freq':220,
                       'Ellip_Taper': -2.1714724,
                       'T_angle_x' : 7.060976,
                       'T_angle_y' : 7.9273612,
                       'Gauss_Taper': -20*np.log10(np.exp(1)),
                       'Gauss_Tangle': 14.121952,
                       'beam_radius': '1.760 mm',
                       'phase_front_radius': '0 mm'},
            '280GHz': {'freq':280,
                       'Ellip_Taper': -2.1714724,
                       'T_angle_x' : 5.7901323,
                       'T_angle_y' : 6.3342926,
                       'Gauss_Taper': -20*np.log10(np.exp(1)),
                       'Gauss_Tangle': 11.58,
                       'beam_radius': '1.686 mm',
                       'phase_front_radius': '0 mm'}
            }


In [2]:
L_lensFp_3 = 1.500000000000000E+000 + 4.500000000000000E-001 + 2.155325400289750E+000 + 5.000000000000000E-002 + 3.100000000000000E-002 +2.570000000000000E-002 + 2.965564711384346E+000
L_lens3_2 = 2.570000000000000E-002 + 3.100000000000000E-002 +5.000000000000000E-002 +1.067669990352706E+001 + 5.000000000000000E-002 +3.100000000000000E-002 +2.570000000000000E-002 +4.696706712699847E+000
L_lens2_1 = 2.570000000000000E-002 + 3.100000000000000E-002 + 5.000000000000000E-002 +3.893300000000000E+000 +4.500000000000000E-001 +5.000000000000000E-001 +4.500000000000000E-001 + 4.777619456395134E+001+ 5.000000000000000E-002+ 3.100000000000000E-002 +2.570000000000000E-002 +4.349908221542306E+000
L_lens1_Lyot = 2.570000000000000E-002 + 3.100000000000000E-002 + 5.000000000000000E-002 +1.055350628144469E+000
L_Ly_vw = 2.94+ 0.3 + 1.06 + 0.36 +0.3706 +0.3548 + 8.1448 + 0.3 + 0.95 +4.0*10**(-4) +0.9 + 4*10**(-4) + 5.53 + 4*10**(-4) + 0.3+ 1.2

In [ ]:
## 1. define coordinate systems
coor_ref = coor_sys([0,0,0],[0,0,0],ref_coor = global_coord,name='coor_feed_ref',)
coor_feed0 = coor_sys([0,0,0],[np.pi,0,0],ref_coor = coor_ref,name='coor_feed0',)
coor_feed1 = coor_sys([0,150,0],[np.pi,0,0],ref_coor = coor_ref,name='coor_feed1',)
coor_lens3 = coor_sys([0,0,-L_lensFp_3*10],[0,0,0],ref_coor = coor_ref,name='coor_lens3',)
coor_lens2 = coor_sys([0,0,-L_lens3_2*10],[0,0,0],ref_coor = coor_lens3,name='coor_lens2',)
#coor_filter1 = coor_sys([0,0,-],[0,0,0],ref_coor = coor_lens2,name='coor_filter1')
coor_lens1 = coor_sys([0,0,-L_lens2_1*10],[0,0,0],ref_coor = coor_lens2,name='coor_lens1',)
coor_Lyot = coor_sys([0,0,-L_lens1_Lyot*10],[0,0,0],ref_coor = coor_lens1,name='coor_Lyot',)
#coor_filter2 = coor_sys([0,0,-804.9559555174151],[0,0,0],ref_coor = coor_ref,name='coor_filter',)
#coor_IR = coor_sys([0,0,-816.9559555174151],[0,0,0],ref_coor = coor_ref,name='coor_IR',)
coor_vw = coor_sys([0,0,-L_Ly_vw*10],[0,0,0],ref_coor = coor_Lyot, name = 'coor_vw')
coor_cut = coor_sys([0,0,0],[np.pi,0,0],ref_coor = coor_ref,name='coor_cut',)
coor_list=[global_coord, coor_ref, 
           coor_feed0, coor_feed1, 
           coor_lens1, coor_lens2, 
           #coor_filter1, 
           coor_lens3,
           coor_Lyot, 
           #coor_filter2, 
           # coor_IR, 
           coor_cut,
           coor_Lyot,
           coor_vw]

In [ ]:
### 2. define lenses
lens1 = simple_lens(coor_lens1, str(2.218520427559085E+001*2)+' cm', 
                 SILICON, loss_tangent = 0, 
                 r1 = '0 cm', r2 = '0 cm', bs1 = 0, bs2 = 0,
                 thickness = '4.349908221542306 cm',
                 surf_f1 = './srf/lens1_f1.rsf',
                 surf_f2 = './srf/lens1_f2.rsf',
                 lengthUnit = 'cm',
                 name='lens1')

lens2 = simple_lens(coor_lens2, str(2.222719209951949E+001*2)+' cm', 
                 SILICON, loss_tangent = 0, 
                 r1 = '0 cm', r2 = '0 cm', bs1 = 0, bs2 = 0,
                 thickness = '4.696706712699847 cm',
                 surf_f1 = './srf/lens2_f1.rsf',
                 surf_f2 = './srf/lens2_f2.rsf',
                 lengthUnit = 'cm',
                 name='lens2')

lens3 = simple_lens(coor_lens3, str(2.005427094320331E+001*2)+' cm', 
                 SILICON, loss_tangent = 0, 
                 r1 = '0 cm', r2 = '0 cm', bs1 = 0, bs2 = 0,
                 thickness = '2.965564711384346 cm',
                 surf_f1 = './srf/lens3_f1.rsf',
                 surf_f2 = './srf/lens3_f2.rsf',
                 lengthUnit = 'cm',
                 name='lens3')

lens_list = [lens1,lens2,lens3]

#### 3 define rim
rim_Lyot = rim([0,0], [210,210], Type = 'elliptical_rim',name='rim_Lyot')
#rim_filter2 = rim([0,0], [213.6294943065297,213.6294943065297], Type = 'elliptical_rim',name='rim_filter2')
#rim_filter1 = rim([0,0], [221.5879439638061,221.5879439638061], Type = 'elliptical_rim',name='rim_filter1')
#rim_IR = rim([0,0], [216.9706978673872,216.9706978673872], Type = 'elliptical_rim',name='rim_IR')
rim_vw = rim([0,0], [2.876618694117068E+001,2.876618694117068E+001], Type = 'elliptical_rim',name='rim_vw')
rim_list = [rim_Lyot, 
            #rim_filter1, rim_filter2, rim_IR,
            rim_vw]

### 3.1 aperture in screen
Lyot = Aperture_screen(coor_Lyot, rim_Lyot,infinity_shadow = 'on', name='Lyot')
#Filter2 = Aperture_screen(coor_filter2, rim_filter2,infinity_shadow = 'on', name='Filter2')
#Filter1 = Aperture_screen(coor_filter1, rim_filter1,infinity_shadow = 'on', name='Filter1')
#IR_absorber = Aperture_screen(coor_IR, rim_IR, infinity_shadow = 'on', name='IR_obsorber')
VW = Aperture_screen(coor_vw, rim_vw,infinity_shadow = 'on', name='vw')
aper_list = [Lyot,
             #Filter2,Filter1,IR_absorber,
             VW]

In [ ]:
## define EO objector
freq_list = frequencyList([Feed_list[freq]['freq']] ,name='freq_list') # Unit GHz
Feed_Gaussian = GaussBeam_Near(freq_list, coor_feed0, 
                               Feed_list[freq]['beam_radius'],Feed_list[freq]['phase_front_radius'],
                               polarisation='linear_x',
                               factor = [0,0],
                               freq_index = 1,
                               name='Gauss_Near_Feed1')

Feed_ellip = Elliptical_Beam(freq_list, coor_feed0, 
                             Feed_list[freq]['Ellip_Taper'], [Feed_list[freq]['T_angle_x'],Feed_list[freq][' T_angle_y']],
                             polarisation='linear', 
                             polarisation_angle=0,
                             far_forced = 'off',
                             factor = [0,0],
                             frequency_index_for_plot = 1,
                             name = 'Gaussian_Elliptical_Beam')


### define PO analysis object
PO_lens1

PO_lens2

PO_lens3

POA_Lyot

PO_lens_VW

POA_VW

In [ ]:
with open('output/SAT_simple.tor','w') as f:
    for item in coor_list:
        f.writelines(item.Str)
    for item in lens_list:
        f.writelines(item.Str)
    for item in rim_list:
        f.writelines(item.Str)
    for item in aper_list:
        f.writelines(item.Str)